<a href="https://colab.research.google.com/github/Jude-Morgan/Machine-Learning-projects/blob/main/TFIDF_text_summerisation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import re
import tensorflow as tf
import string
import seaborn as sns
import textwrap

In [ ]:
#Import natural language toolkit
import nltk
from nltk.corpus import stopwords
nltk.download('stop_words')
nltk.download('punkt')
from nltk import sent_tokenize, word_tokenize, PorterStemmer
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import SmoothingFunction
from nltk.translate.bleu_score import sentence_bleu
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

[nltk_data] Error loading stop_words: Package 'stop_words' not found
[nltk_data]     in index
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
!pip install contractions

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
data = pd.read_csv("/content/drive/MyDrive/Airplane_Seat_Article.csv", encoding = "ISO-8859-1")
#Print dataframe
data

,article_id,article,highlights
0,1,Ever noticed how plane seats appear to be gett...,Experts question if packed out planes are put...


In [ ]:
print(data.columns)

Index(['article_id', 'article', 'highlights'], dtype='object')


In [ ]:
import re
import contractions

def clean_text(text):
#Cleans text by removing noise and formatting
  #Remove URLs
  text = re.sub(r'https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)
  #Remove HTML tags
  text = re.sub(r'\<a href', ' ', text)
  #Remove ampersands
  text = re.sub(r'&amp;', ' ', text)
  #Remove special characters
  text = re.sub(r'[_\-;%()|+&=*%:#$@\[\]/]', ' ', text)
  #Remove line breaks
  text = re.sub(r'<br />', ' ', text)
  #Remove single quotes
  text = re.sub(r'\'', ' ', text)
  #Remove newlines
  text = re.sub(r'\n', ' ', text)
  #Remove est
  text = re.sub(' est ',' ', text)
  #Replace question marks and exclamation points with periods
  text = re.sub(r'[?!]', '.', text)
  return text

#Expands contractions in text
def expand_contractions(text):
  text = text.split()
  final = []
  for word in text:
    try:
      final.append(contractions.fix(word) + " ")
    except:
      final.append(word + " ")
  return "".join(final)

#Removes sentences from a list of texts that are less than 5 words long
def remove_short_sentences(texts):
  final = []
  for text in texts:
    sents = []
    sentences = text.split(".")
    for sentence in sentences:
      if len(sentence.split()) >= 5:
        sents.append(sentence + ".")
    final.append("".join(sents))
  return final

#Removes the CNN and est tags from a list of texts
def remove_tag(texts):
  final = []
  for text in texts:
    cnn = text.find("cnn")
    if cnn != -1 and cnn < len(text) // 10:
      text = text[cnn + 3:]
    found = False
    for i in range(2):
      est = text.find(" est,")
      if est < len(text) // 5 and est != -1:
        text = text[est + 5:]
        found = True
    fs = text.find(".")
    if fs < 20 and fs != -1 and found:
      text = text[fs:]
    final.append(text)
  return final

#Preprocesses text by cleaning it, expanding contractions, removing short sentences, and removing the CNN and est tags
def preprocess_text(text):
  text = clean_text(text)
  text = expand_contractions(text)
  text = remove_short_sentences([text])[0]
  text = remove_tag([text])[0]
  return text

In [ ]:
texts = data.iloc[:, 1]
new_texts = []
for text in texts:
  new_texts.append(expand_contractions(clean_text(text)).lower())
final_cleaned = remove_tag(remove_short_sentences(new_texts))

In [ ]:
summaries = data.iloc[:, 2]
new_summaries = []
for summary in summaries:
  new_summaries.append(expand_contractions(clean_text(summary)).lower())

In [ ]:
#Create new dataframe with preprocessed texts and summaries
new_data = pd.DataFrame()
new_data["Texts"] = final_cleaned
new_data["Summaries"] = new_summaries

In [ ]:
new_data.to_csv("Cleaned.csv")
new_data

,Texts,Summaries
0,ever noticed how plane seats appear to be gett...,experts question if packed out planes are putt...


In [ ]:
print(new_data.columns)

Index(['Texts', 'Summaries'], dtype='object')


In [ ]:
new_data.columns = new_data.columns.astype(str)
print(new_data.columns)

Index(['Texts', 'Summaries'], dtype='object')


In [ ]:
#Returns a set of all the words in the given texts
def words(texts: list) -> set:
    import re
    all_words = set()
    for text in texts:
        text = re.sub(r"[.,]", " ", text)
        all_words |= set(text.split())
    return all_words

final_cleaned = ["This is a sentence. This is another sentence."]
swords = words(final_cleaned)

In [ ]:
#Returns a dictionary of term frequencies for the given sentences
def create_frequency_matrix(sentences: list) -> dict:
    frequency_matrix = {}
    stop_words = set(stopwords.words("english"))
    stemmer = PorterStemmer()

    for sentence in sentences:
        frequency_table = {}
        words = word_tokenize(sentence)
        for word in words:
            word = word.lower()
            word = stemmer.stem(word)
            if word in stop_words:
                continue

            if word in frequency_table:
                frequency_table[word] += 1
            else:
                frequency_table[word] = 1

        frequency_matrix[sentence[:15]] = frequency_table

    return frequency_matrix

In [ ]:
#Returns a dictionary of term frequencies for the given sentences
def create_tf_matrix(frequency_matrix: dict) -> dict:
    tf_matrix = {}

    for sentence, frequency_table in frequency_matrix.items():
        tf_table = {}

        count_words_in_sentence = len(frequency_table)
        for word, count in frequency_table.items():
            tf_table[word] = count / count_words_in_sentence

        tf_matrix[sentence] = tf_table

    return tf_matrix

In [ ]:
#Returns a dictionary of the number of documents each word appears in
def create_documents_per_words(frequency_matrix: dict) -> dict:
    word_per_document_table = {}

    for sentence, frequency_table in frequency_matrix.items():
        for word, count in frequency_table.items():
            if word in word_per_document_table:
                word_per_document_table[word] += 1
            else:
                word_per_document_table[word] = 1

    return word_per_document_table

In [ ]:
#Returns a dictionary of inverse document frequencies for the given sentences
def create_idf_matrix(frequency_matrix: dict, documents_per_word_table: dict, total_documents: int) -> dict:
    idf_matrix = {}

    for sentence, frequency_table in frequency_matrix.items():
        idf_table = {}

        for word in frequency_table.keys():
            idf_table[word] = math.log10(total_documents / float(documents_per_word_table[word]))

        idf_matrix[sentence] = idf_table

    return idf_matrix

In [ ]:
#Returns a dictionary of term frequency-inverse document frequency for the given sentences
def create_tf_idf_matrix(tf_matrix: dict, idf_matrix: dict) -> dict:
    tf_idf_matrix = {}

    for (sentence1, tf_table1), (sentence2, tf_table2) in zip(tf_matrix.items(), idf_matrix.items()):
        tf_idf_table = {}

        for (word1, value1), (word2, value2) in zip(tf_table1.items(), tf_table2.items()):
            tf_idf_table[word1] = float(value1 * value2)

        tf_idf_matrix[sentence1] = tf_idf_table

    return tf_idf_matrix

In [ ]:
#score a sentence by its word's TF Basic algorithm: adding the TF frequency of every non-stop word in a sentence divided by total no of words in a sentence.

def score_sentences(tf_idf_matrix: dict) -> dict:
    sentence_scores = {}

    for sentence, tf_idf_table in tf_idf_matrix.items():
        total_score_per_sentence = 0.0

        count_words_in_sentence = len(tf_idf_table)
        for word, score in tf_idf_table.items():
            total_score_per_sentence += score

        sentence_scores[sentence] = total_score_per_sentence / count_words_in_sentence

    return sentence_scores

In [ ]:
#Returns the average score of sentences from the sentence scores dictionary
def find_average_score(sentence_scores: dict) -> float:
    sum_of_scores = 0.0
    for sentence, score in sentence_scores.items():
        sum_of_scores += score

    average_score = sum_of_scores / len(sentence_scores)

    return average_score

In [ ]:
#Returns a summary of the given sentences based on their sentence scores and the given threshold
def generate_summary_TFIDF(sentences: list, sentence_scores: dict, threshold: float) -> str:
    sentence_count = 0
    summary = ""

    for sentence in sentences:
        if sentence[:15] in sentence_scores and sentence_scores[sentence[:15]] >= threshold:
            summary += " " + sentence
            sentence_count += 1

    return summary

In [ ]:
import math

#Sentence Tokenize
sentences = sent_tokenize(text)
total_documents = len(sentences)
#print(sentences)

#Create the Frequency matrix of the words in each sentence.
freq_matrix = create_frequency_matrix(sentences)
#print(freq_matrix)

#Calculate TermFrequency and generate a matrix
tf_matrix = create_tf_matrix(freq_matrix)
#print(tf_matrix)

#Creating table for documents per words
count_doc_per_words = create_documents_per_words(freq_matrix)
#print(count_doc_per_words)

#Calculate IDF and generate a matrix
idf_matrix = create_idf_matrix(freq_matrix, count_doc_per_words, total_documents)
#print(idf_matrix)

#Calculate TF-IDF and generate a matrix
tf_idf_matrix = create_tf_idf_matrix(tf_matrix, idf_matrix)
#print(tf_idf_matrix)

#Important Algorithm: score the sentences
sentence_scores = score_sentences(tf_idf_matrix)
#print(sentence_scores)

#Find the threshold
threshold = find_average_score(sentence_scores)
#print(threshold)

#Generate summary
summary = generate_summary_TFIDF(sentences, sentence_scores, 1.3 * threshold)
print(summary)

 Ever noticed how plane seats appear to be getting smaller and smaller? The distance between two seats from one point on a seat to the same point on the seat behind it is known as the pitch.


In [ ]:
from nltk.tokenize import word_tokenize
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

#Generate summary
generated_summary = generate_summary_TFIDF(sentences, sentence_scores, 1.3 * threshold)

#Tokenize the generated summary and reference summaries
generated_summary_tokens = word_tokenize(generated_summary)
reference_summaries_tokens = [word_tokenize(summary) for summary in new_data["Summaries"]]

#Calculate BLEU score
bleu_score = sentence_bleu(reference_summaries_tokens, generated_summary_tokens, smoothing_function=SmoothingFunction().method1)

#Print BLEU score
print("BLEU Score:", bleu_score)

BLEU Score: 0.006243702571093031
